In [ ]:
import ollama

# 1. 사전 정의된 카테고리
categories = [
    "식품 및 음료", "의류 및 패션 액세서리", "가전제품 및 전자기기",
    "생활용품 및 청소용품", "미용 및 퍼스널 케어", "가구 및 인테리어",
    "도서 및 문구", "스포츠 및 레저", "완구 및 장난감", "반려동물 용품"
]

# 2. 분류할 상품 리스트
products = [
    "미식상회 저온압착 참기름 및 모든 제품 최저가 판매 미식상회 60년 전통 100% 통참깨 참기름/들기름 350ml 후기대박 전통방식",
    "가시제로고등어 톡딜 초특가 가시 발견시 환불! 피터씨마켓 가시제로 노르웨이 고등어 플러스 900g x 2",
    "더미식 즉석밥&덮밥소스 교자&국물류까지 최대할인! 더미식 당진쌀로 만든 당진 백미밥 200g 24개 외 잡곡밥류,덮밥소스 모음",
    "믿고쓰는 친환경인증 피톤치드 숲향기 그대로! 항균 탈취 친환경인증 편백나무 피톤치드 스프레이 원액 차량 자동차 탈취제 새집증후군 담배냄새 없애는방법 냄새제거",
    "캉골 카쇼라 런칭특집! 신학기맞이 백팩/도트백 등 캉골 신학기 신상 가방 모음전 톡딜가109,000원~",
    "동원 부채교 딤섬 신제품출시 속이 보이는 살아있는 식감 [동원] 딤섬 부채교/새우하가우/부추창펀/샤오롱바오 6봉 골라담기 (4봉+2봉) 만두",
]

# 3. Ollama를 사용한 카테고리 분류 함수
def classify_product(product):
    prompt = f"""
    다음 상품 설명을 가장 적절한 카테고리로 분류해 주세요.

    상품 설명: "{product}"

    가능한 카테고리 목록:
    {', '.join(categories)}

    적절한 카테고리 이름만 출력하세요.
    """
    response = ollama.chat(model="EEVE-Korean-10.8B", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']

# 4. 모든 상품을 분류
for product in products:
    category = classify_product(product)
    print(f"상품: {product[:50]}... → 카테고리: {category}")
